#1.importing libraries

In [10]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
#printing the stopwords in english language
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

#2.preprocessing


1.framing the data into data frame

In [13]:
#loading the data from csv file to pandas data frame
twitter_data = pd.read_csv('/content/twitter_training.csv',encoding= 'ISO-8859-1')

ParserError: Error tokenizing data. C error: EOF inside string starting at row 39308

In [ ]:
#printing the no of rows and columns
twitter_data.shape

In [ ]:
#printing the first five rows
twitter_data.head()

2.data managing

In [ ]:
#naming the columns
column_names=['id','flag','target','text']
twitter_data = pd.read_csv('/content/twitter_training.csv', names = column_names, encoding= 'ISO-8859-1')

In [ ]:
twitter_data.shape

In [ ]:
twitter_data.head()

In [ ]:
#counting the no of missing values
twitter_data.isnull().sum()

In [ ]:
twitter_data=twitter_data.dropna()
twitter_data.isna().sum()

In [ ]:
#checking the distribution of target column
twitter_data['target'].value_counts()

3.stemming


In [ ]:
port_stem = PorterStemmer()

In [ ]:
def preprocess_and_tokenize(content):
    # Removing non-alphabet characters and converting to lowercase
    content = re.sub('[^a-zA-Z]', ' ', content).lower()
    # Splitting and removing stopwords
    tokens = [port_stem.stem(word) for word in content.split() if word not in stopwords.words('english')]
    return tokens

# Tokenize the tweets
twitter_data['tokens'] = twitter_data['text'].apply(preprocess_and_tokenize)



In [ ]:
from sklearn.preprocessing import LabelEncoder
# Assuming your target column is named 'target'
label_encoder = LabelEncoder()
twitter_data['target'] = label_encoder.fit_transform(twitter_data['target'])

# Check the unique values after encoding
print("Encoded classes:", label_encoder.classes_)

In [ ]:
# Separating the data and labels
x = twitter_data['tokens'].values
y = twitter_data['target'].values

In [ ]:
twitter_data.head()

In [ ]:
print(x)

In [ ]:
print(y)

#3.Splitting the data into training data and test data



In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)

In [ ]:
print(x.shape,x_train.shape,x_test.shape)

4.vectorization


In [ ]:
import gensim
from gensim.models import Word2Vec

In [ ]:
#5. Word2Vec Embeddings
# Training Word2Vec model on the training data
w2v_model = Word2Vec(sentences=x_train.tolist(), vector_size=100, window=5, min_count=1, sg=1)

# Function to get the average word vector for each tweet
def get_average_word2vec(tokens, model, vector_size):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(vector_size)

# Creating average word vectors for each tweet in training and test sets
x_train_vec = np.array([get_average_word2vec(tokens, w2v_model, 100) for tokens in x_train])
x_test_vec = np.array([get_average_word2vec(tokens, w2v_model, 100) for tokens in x_test])

# 4.Training the Machine learning model

Logistic Regression

In [ ]:
lr = LogisticRegression()
lr.fit(x_train_vec,y_train)

In [ ]:
x_train_prediction = lr.predict(x_train_vec)
training_data_accuracy = accuracy_score(x_train_prediction,y_train)

In [ ]:
print('Accuracy score of the training data: ',training_data_accuracy*100)

In [ ]:
import xgboost as xgb
clf_xgb=xgb.XGBClassifier()
clf_xgb.fit(x_train_vec,y_train)
x_train_prediction_xgb=clf_xgb.predict(x_train_vec)
training_data_accuracy_xgb=accuracy_score(x_train_prediction_xgb,y_train)
print(training_data_accuracy_xgb*100)

In [ ]:

from sklearn.ensemble import RandomForestClassifier
#random forest model
rf_model = RandomForestClassifier()
rf_model.fit(x_train_vec, y_train)

In [ ]:
x_train_prediction_rf = rf_model.predict(x_train_vec)
accuracy_rf = accuracy_score(y_train, x_train_prediction_rf)
#print(f'Accuracy of Random Forest model: {accuracy_rf:.2f}')
print(accuracy_rf*100)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Assuming x_train_vec is your feature matrix with potential negative values
scaler = MinMaxScaler()
x_train_vec_scaled = scaler.fit_transform(x_train_vec)
x_test_vec_scaled = scaler.transform(x_test_vec)

MN Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
# Now fit the MultinomialNB model
classifier = MultinomialNB()
classifier.fit(x_train_vec_scaled, y_train)

In [ ]:

x_train_prediction_nb = classifier.predict(x_train_vec_scaled)
training_datas_accuracy_nb = accuracy_score(x_train_prediction_nb,y_train)

In [ ]:
print('Accuracy score of the training data: ',training_datas_accuracy_nb*100)

In [ ]:
from sklearn.ensemble import VotingClassifier
ensemble_model = VotingClassifier(estimators=[
    ('logistic_regression', lr),
    ('xgboost', clf_xgb),
    ('naive_bayes', classifier),
    ('randomforest', rf_model)
], voting='hard')  # Use 'soft' for probability-based voting

# Train the ensemble model on the training data
ensemble_model.fit(x_train_vec_scaled, y_train)

# Evaluate the ensemble model on training data
y_train_pred_ensemble = ensemble_model.predict(x_train_vec_scaled)
training_accuracy_ensemble = accuracy_score(y_train_pred_ensemble, y_train)
print('Ensemble Model Accuracy on Training Data: ', training_accuracy_ensemble * 100)

In [ ]:
# Plotting training accuracies

import seaborn as sns
import matplotlib.pyplot as plt

models = ['Logistic Regression', 'XGBoost', 'Naive Bayes', 'RandomForest']
accuracies = [
    training_data_accuracy * 100,
    training_data_accuracy_xgb * 100,
    training_datas_accuracy_nb * 100,
    accuracy_rf * 100
]

plt.figure(figsize=(10, 6))
sns.barplot(x=models, y=accuracies)
plt.title('Training Accuracy of Different Models')
plt.xlabel('Models')
plt.ylabel('Accuracy (%)')
plt.ylim(0, 100)
plt.show()